In [1]:
import tabula
import pandas as pd
import numpy as np
from tabula import read_pdf
import PyPDF2
import re
import pickle
import functools
import itertools
import operator

In [4]:
class pdf_scraping:
    
    def __init__(self,crd):
        self.crd = crd
    
    def get_pagenum(self):
        # open the pdf file
        pdfFileObject = open(str(self.crd) + '.pdf', 'rb')
        Object = PyPDF2.PdfFileReader(pdfFileObject)
        # get number of pages
        NumPages = Object.getNumPages()
        return NumPages

    def show_df(self,page):
        if page <= self.get_pagenum():
            f = tabula.read_pdf(str(self.crd) + '.pdf', pages = page, multiple_tables = True, 
                                guess = False, stream = True)[0]
            return f
        else:
            print("The page number you entered exceed the max page number in the PDF, reenter a smaller page number")

    def count_numOf_privateFund(self):
        pageNum = self.get_pagenum()
        if pageNum <= 25:
            Page_Table = tabula.read_pdf(str(self.crd) + '.pdf', pages = min(25,pageNum), 
                            multiple_tables = True,guess = False, stream=True)[0]
        
        else:
            temp_table_1 = tabula.read_pdf(str(self.crd) + '.pdf', pages = min(25,pageNum), 
                            multiple_tables = True,guess = False, stream=True)[0]

            temp_table_2 = tabula.read_pdf(str(self.crd) + '.pdf', pages = pageNum, 
                                        multiple_tables = True,guess = False, stream=True)[0]

            temp_table_2 = temp_table_2.rename(columns={temp_table_2.columns[0]: "FORM ADV"})
            Page_Table = pd.concat([temp_table_1,temp_table_2[[temp_table_2.columns[0]]]],axis = 0).reset_index(drop = True)
        

        
        Private_Fund_Reporting_indexes = Page_Table[Page_Table['FORM ADV'].str.contains(r'11. Current gross asset value of the private fund',na=False)].index
        private_fund_leading_indexes = Page_Table[Page_Table['FORM ADV'] == 'A. PRIVATE FUND'].index

        
        count = min(len(private_fund_leading_indexes),len(Private_Fund_Reporting_indexes))
        if count == 0:
            return count, np.nan
        else:
            #This is a list to store all the useful information we highlighted on PDF
            #count>0, in this case the company must have some information in this section
            list_for_meta_privateFund = []
            
            for i in range(count):
                #list: [Name of the private fund,Privatefundidentificationnumber,gross_asset_value]
                l = [Page_Table['FORM ADV'].loc[private_fund_leading_indexes[i] + 3],
                    Page_Table['FORM ADV'].loc[private_fund_leading_indexes[i] + 6],
                    Page_Table['FORM ADV'].loc[Private_Fund_Reporting_indexes[i] + 1]]
                list_for_meta_privateFund.append(l)
            return count, list_for_meta_privateFund
    
   
    @classmethod
    def Private_fund_dictionary(cls,start,end):
        
        with open ('/Users/guapofish/crdlist_FILE', 'rb') as fp:
            l = pickle.load(fp)
        
        #count = 0
        dic = {}
        additional_info_col = []
        missing_crd = []
        for item in l[start:end]:            
            try:
                s = cls(item) 
                num, dic_for_meta_prFund = s.count_numOf_privateFund()
                dic[str(item)] = int(num) #key: crd number, value: num of private fund
                additional_info_col.append(dic_for_meta_prFund)
            except:
                missing_crd.append(item)
                additional_info_col.append(np.nan)
                print("Error: crd number {} ".format(item))
                
        
        output_df = pd.DataFrame(dic.items(),columns=['CRD_Number', 'Number of Private Fund'])
        output_df['Private Fund Infomation'] = additional_info_col
        return output_df
    
    @classmethod
    def get_more_private_fund_info(cls,start,end):

        df = cls.Private_fund_dictionary(start,end)
        df_new = df[df['Number of Private Fund'] != 0]

        max_column = df_new['Number of Private Fund'].max()
        new_column_1 = ['Private Fund {} Name'.format(i) for i in range(1,max_column+1)]
        new_column_2 = ['Private Fund {} Id number'.format(i) for i in range(1,max_column+1)]
        new_column_3 = ['Private Fund {} gross asset value'.format(i) for i in range(1,max_column+1)]
        zip_col = zip(new_column_1, new_column_2, new_column_3)
        zip_column = functools.reduce(operator.iconcat, zip_col, [])

        df_new = df_new.reindex(columns=[*df_new.columns.tolist(), *zip_column], fill_value=np.nan)
        idx = df_new.index
        
        for i in range(len(df_new)):
            Number_of_fund = int(df_new['Number of Private Fund'].iloc[i])
            list2d = df_new['Private Fund Infomation'].iloc[i]
            merged = list(itertools.chain.from_iterable(list2d))
            col_indexer = df_new.columns[3: 3* (Number_of_fund + 1)]
            df_new.loc[idx[i],col_indexer] = merged

        return df, df_new
    
    

In [5]:
df_all, df_with_private_fund = pdf_scraping.get_more_private_fund_info(0,100)

In [8]:
df_with_private_fund.head()

,CRD_Number,Number of Private Fund,Private Fund Infomation,Private Fund 1 Name,Private Fund 1 Id number,Private Fund 1 gross asset value,Private Fund 2 Name,Private Fund 2 Id number,Private Fund 2 gross asset value,Private Fund 3 Name,Private Fund 3 Id number,Private Fund 3 gross asset value,Private Fund 4 Name,Private Fund 4 Id number,Private Fund 4 gross asset value,Private Fund 5 Name,Private Fund 5 Id number,Private Fund 5 gross asset value
0,133693,1,"[[H/2 CORE REAL ESTATE DEBT FUND L.P., 805-154...",H/2 CORE REAL ESTATE DEBT FUND L.P.,805-1546899628,"$ 15,075,593",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,105957,3,"[[ALBION FUND I, LP, 805-1842512925, $ 1,412,3...","ALBION FUND I, LP",805-1842512925,"$ 1,412,342","ALBION FUND II, LP",805-1237420933,"$ 8,819,355","ALBION FUND III, LP",805-5162809993,"$ 5,411,353",NaN,NaN,NaN,NaN,NaN,NaN
5,159529,4,"[[ALTUS CAPITAL PARTNERS II, L.P., 805-8237362...","ALTUS CAPITAL PARTNERS II, L.P.",805-8237362824,"$ 111,898,572","ALTUS CAPITAL PARTNERS III, L.P.",805-1660019630,"$ 68,380,407","ALTUS CAPITAL PARTNERS PARENT, L.P.",805-4173471674,"$ 9,741,427","ALTUS-CHOICE CO-INVESTMENT, LLC",805-9123190464,"$ 3,823,000",NaN,NaN,NaN
8,299599,1,"[[STAMINA MASTER FUND LP, 805-6692011350, $ 29...",STAMINA MASTER FUND LP,805-6692011350,"$ 290,458,246",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,162954,3,"[[TDF VENTURES FUND IV, LP, 805-8769684196, $ ...","TDF VENTURES FUND IV, LP",805-8769684196,"$ 88,941,904","TDFUND II, LP",805-7129926617,"$ 2,521,166","TDFUND III, LP",805-6003965942,"$ 72,284,051",NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_all.head()

,CRD_Number,Number of Private Fund,Private Fund Infomation
0,133693,1,"[[H/2 CORE REAL ESTATE DEBT FUND L.P., 805-154..."
1,282351,0,NaN
2,145800,0,NaN
3,285650,0,NaN
4,105957,3,"[[ALBION FUND I, LP, 805-1842512925, $ 1,412,3..."


In [10]:
df_all['Number of Private Fund'].value_counts()

0    67
1    11
2    10
3     8
5     2
4     2
Name: Number of Private Fund, dtype: int64

In [12]:
#df_all.to_csv('/Users/guapofish/Desktop/Deer Isle Group/df_all_100.csv')
#df_with_private_fund.to_csv('/Users/guapofish/Desktop/Deer Isle Group/df_with_private_fund_100.csv')